In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv
from operator import itemgetter 
import os

from tqdm import tqdm

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [10]:
def load_vdb_and_retriever(path="../vectorstore/historia_ed_financeira",
                           k=4):
    embedding_size = 1536
    embedding_model = "text-embedding-3-small"
    embeddings = OpenAIEmbeddings(model=embedding_model, dimensions=embedding_size)
    
    vdb = FAISS.load_local(path, 
                           embeddings, 
                           allow_dangerous_deserialization=True)
    
    retriever = vdb.as_retriever(search_kwargs={"k": k})
    
    return vdb, retriever
    

### Chains (Requisições para LLMs)

Para fazer as requisições para os LLMs, vamos utilizar a biblioteca ``LangChain``. Ela é uma biblioteca que permite fazer requisições para LLMs de forma mais simples e rápida.

### RAG (Retrieval-Augmented Generation)

A técnica utilizada aqui consiste em utilizar o mecanismo de busca baseado nos ``embeddings`` para que se encontre os documentos mais próximos semanticamente para a geração de texto. Dessa forma, o modelo consegue gerar textos que se baseiam na informação contida nos documentos mais próximos.

<img src="../imgs/rag.png" width="600">

In [11]:
vdb, retriever = load_vdb_and_retriever()

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
# llm = ChatGroq(model="llama-3.3-70b-versatile", temperature=0)


system_prompt = """
Você é um assistente de IA que vai tirar dúvidas sobre educação financeira e história. 

Além disso, aqui está um conteudo extra sobre educação financeira e/ou história:

[Conteudo extra]
{extra_content}
[Final do conteudo extra]

--------------------------------------------
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

chain = (
    {
        "input": itemgetter("input"),
        "extra_content": itemgetter("input") | retriever,
    }
    | prompt 
    | llm 
    | StrOutputParser()
)

In [13]:
response = chain.invoke({"input": "Me fale sobre o peso de um sonho"})

In [14]:
print(response)

O conceito de "peso de um sonho" pode ser entendido de várias maneiras, mas geralmente se refere à importância e ao impacto que um sonho ou objetivo tem na vida de uma pessoa. Assim como na analogia do quilo de algodão e o quilo de chumbo, que pesam a mesma coisa, mas têm diferentes percepções, os sonhos também podem ser percebidos de maneiras distintas.

Quando falamos sobre o "peso" de um sonho, podemos considerar:

1. **Prioridade**: Alguns sonhos podem ser mais importantes do que outros, dependendo das circunstâncias da vida de uma pessoa. É essencial identificar quais sonhos são prioritários e quais podem ser adiados.

2. **Esforço e Sacrifício**: Realizar um sonho muitas vezes requer esforço, dedicação e, em muitos casos, sacrifícios financeiros e pessoais. O "peso" pode se referir ao quanto você está disposto a se esforçar para alcançá-lo.

3. **Impacto Emocional**: O peso de um sonho também pode estar relacionado ao impacto emocional que ele tem. Sonhos que são profundamente si

In [15]:
async for chunk in chain.astream({"input": "Me fale sobre o peso de um sonho"}):
    print(chunk, end="", flush=True)

O conceito de "peso de um sonho" pode ser entendido de várias maneiras, mas geralmente se refere à importância e ao impacto que um sonho ou objetivo tem na vida de uma pessoa. Assim como na analogia do quilo de algodão e o quilo de chumbo, que pesam a mesma coisa, mas têm diferentes percepções, os sonhos também podem ser percebidos de maneiras distintas.

1. **Importância Pessoal**: O peso de um sonho pode ser medido pela sua relevância para a vida de alguém. Sonhos que estão alinhados com os valores e desejos mais profundos de uma pessoa tendem a ter um "peso" maior, pois são motivadores poderosos.

2. **Desafios e Sacrifícios**: Alguns sonhos podem exigir sacrifícios significativos, como tempo, dinheiro e esforço. O "peso" aqui se refere ao que a pessoa está disposta a investir para realizá-lo. Sonhos que demandam mais esforço podem parecer mais pesados.

3. **Realização e Satisfação**: O peso de um sonho também pode ser visto na satisfação que sua realização traz. Sonhos que são alc